In [1]:
import sqlite3
import csv
import pandas as pd

Connect to sqlite3 database

In [2]:
conn = sqlite3.connect('S30 ETL Assignment.db')
cursor = conn.cursor()

Executing sql query to fetch output

In [3]:
cursor.execute("""
    SELECT c.customer_id, c.age, i.item_name, SUM(COALESCE(o.quantity, 0)) AS total_quantity
    FROM customers c
    LEFT JOIN sales s ON c.customer_id = s.customer_id
    LEFT JOIN orders o ON s.sales_id = o.sales_id
    LEFT JOIN items i ON o.item_id = i.item_id
    WHERE c.age BETWEEN 18 AND 35
    GROUP BY c.customer_id, i.item_id
    HAVING total_quantity > 0
    ORDER BY c.customer_id, i.item_id
""")

rows = cursor.fetchall()

In [4]:
rows

[(5, 18, 'x', 3),
 (5, 18, 'y', 3),
 (5, 18, 'z', 4),
 (7, 30, 'x', 14),
 (7, 30, 'y', 19),
 (7, 30, 'z', 10),
 (9, 25, 'x', 5),
 (9, 25, 'y', 2),
 (9, 25, 'z', 4),
 (16, 19, 'x', 5),
 (16, 19, 'y', 4),
 (16, 19, 'z', 8),
 (22, 18, 'x', 5),
 (22, 18, 'y', 5),
 (22, 18, 'z', 3),
 (28, 21, 'x', 3),
 (28, 21, 'z', 1),
 (29, 30, 'x', 5),
 (29, 30, 'y', 12),
 (29, 30, 'z', 12),
 (31, 22, 'x', 6),
 (31, 22, 'y', 7),
 (31, 22, 'z', 12),
 (34, 22, 'x', 5),
 (34, 22, 'y', 6),
 (34, 22, 'z', 5),
 (35, 20, 'x', 3),
 (35, 20, 'y', 5),
 (35, 20, 'z', 2),
 (41, 21, 'x', 2),
 (41, 21, 'y', 4),
 (41, 21, 'z', 6),
 (50, 31, 'y', 2),
 (50, 31, 'z', 1),
 (58, 30, 'x', 5),
 (58, 30, 'y', 2),
 (58, 30, 'z', 4),
 (59, 24, 'x', 7),
 (59, 24, 'y', 4),
 (59, 24, 'z', 5),
 (68, 32, 'x', 5),
 (68, 32, 'y', 5),
 (68, 32, 'z', 3),
 (70, 29, 'x', 5),
 (70, 29, 'y', 6),
 (70, 29, 'z', 1),
 (78, 19, 'x', 9),
 (78, 19, 'y', 8),
 (78, 19, 'z', 3),
 (79, 26, 'x', 5),
 (79, 26, 'y', 4),
 (79, 26, 'z', 7),
 (80, 23, 'x', 

Writing the result to a CSV file

In [5]:
with open('SQL_output.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Customer', 'Age', 'Item', 'Quantity'])
    for row in rows:
        csv_writer.writerow(row)

Loading all table into pandas dataframe

In [6]:
customers_df = pd.read_sql_query("SELECT * FROM customers", conn)
sales_df = pd.read_sql_query("SELECT * FROM sales", conn)
orders_df = pd.read_sql_query("SELECT * FROM orders", conn)
items_df = pd.read_sql_query("SELECT * FROM items", conn)

Merging table to get required data

In [7]:
merged_df = customers_df.merge(sales_df, on='customer_id', how='left') \
                        .merge(orders_df, on='sales_id', how='left') \
                        .merge(items_df, on='item_id', how='left')

Pandas code to get desired output

In [8]:
result_df = merged_df[(merged_df['age'] >= 18) & (merged_df['age'] <= 35)] \
                .groupby(['customer_id', 'age', 'item_name'])['quantity'] \
                .sum() \
                .reset_index()

In [9]:
result_df = result_df[result_df['quantity'] > 0]

In [10]:
result_df

,customer_id,age,item_name,quantity
0,5,18,x,3.0
1,5,18,y,3.0
2,5,18,z,4.0
3,7,30,x,14.0
4,7,30,y,19.0
...,...,...,...,...
67,97,23,y,2.0
68,97,23,z,3.0
69,100,30,x,1.0
70,100,30,y,1.0


Write the result to a csv file

In [11]:
result_df.to_csv('Pandas_output.csv', index=False)

Closing the database connection

In [12]:
conn.close()